In [1]:
from model import ModelConfig, LlamaModel
from train import TrainerConfig, DataLoader, Trainer

from transformers import AutoTokenizer

In [2]:
tokenizer_id = "HuggingFaceTB/SmolLM2-135M"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.pad_token = tokenizer.eos_token


In [4]:
model_config = ModelConfig(
    vocab_size=tokenizer.vocab_size,
    d_model=576,
    d_head=64,
    d_mlp_proj=1536,
    n_layers=30,
    n_kv_heads=3,
    n_attn_heads=9,
    rms_norm_eps=1e-5,
    initializer_range=0.041666666666666664,
    rope_theta=100000.0,
    padding_idx=tokenizer.pad_token_id
)

In [5]:
train_config = TrainerConfig(
    per_device_train_batch_size=8,
    max_seq_len=2048,
    num_epochs=20,
    learning_rate=5e-4
)

In [6]:
with open("data/tiny_shakespeare.txt") as f:
    text = f.read()

In [7]:
model = LlamaModel(model_config)
dataloader = DataLoader(train_config, tokenizer, text)
trainer = Trainer(train_config, model)

Num Trainable Params                     | 162,826,560
Train device                             | cuda, NVIDIA GeForce RTX 3090, N=1
Training precision                       | torch.bfloat16
Flash Attention                          | True
torch.compile()                          | True





In [ ]:
trainer.train(dataloader)

Training steps                           | 420 
Step: 0, Training Loss: 11.33942, Tokens/sec: 1508.647695708986
Step: 1, Training Loss: 9.88539, Tokens/sec: 1637.9013713920879
Step: 2, Training Loss: 11.45460, Tokens/sec: 88256.45896240971
Step: 3, Training Loss: 9.63782, Tokens/sec: 85162.02327040584
Step: 4, Training Loss: 9.43866, Tokens/sec: 84547.14615448186
Step: 5, Training Loss: 9.14640, Tokens/sec: 82961.51713678939
Step: 6, Training Loss: 8.82494, Tokens/sec: 84218.94041861851
Step: 7, Training Loss: 8.43781, Tokens/sec: 83195.84066252301
Step: 8, Training Loss: 8.00212, Tokens/sec: 83471.24335826166
Step: 9, Training Loss: 7.66222, Tokens/sec: 84446.60527311619
Step: 10, Training Loss: 7.35020, Tokens/sec: 82258.91867250211
Step: 11, Training Loss: 7.26622, Tokens/sec: 85034.19896553642
Step: 12, Training Loss: 7.17824, Tokens/sec: 85252.81205138942
Step: 13, Training Loss: 7.04718, Tokens/sec: 83933.51551627144
Step: 14, Training Loss: 6.95707, Tokens/sec: 85043.77654943014

In [ ]:
input_ids = tokenizer(["HAMLET:\nTo be or"], return_tensors="pt")['input_ids'].to(trainer.device)
idx = model.generate(input_ids, temperature=0.25, top_k=50, max_new_tokens=64)
print(tokenizer.batch_decode(idx)[0])